In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install bokeh

In [3]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()

Loading BokehJS ...

In [4]:
dataset = pd.read_csv("pumf-98M0001-E-2016-individuals_F1.csv")
dataset.head()

,PPSORT,WEIGHT,WT1,WT2,WT3,WT4,WT5,WT6,WT7,WT8,...,Subsidy,Tenur,TotInc,TotInc_AT,VALUE,VisMin,Wages,WKSWRK,WRKACT,YRIMM
0,453141,37.037277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9,1,97000,73000,450000,13,95000,6,11,9999
1,923226,37.037277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9,1,99999999,99999999,440000,13,99999999,9,99,9999
2,385097,37.037277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9,1,99999999,99999999,440000,13,99999999,9,99,9999
3,732612,37.037277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,592.596438,...,9,1,46000,41000,839779,13,19000,6,11,9999
4,143665,37.120914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,9,1,30000,26000,60000,13,29000,5,9,9999


###  Notes from Mr. Fogel

MTNNO (Mother Tongue - first write-in component) and HLANO (Home Language - first write-in component) - see if these match, use boolean whether they match as classification. Also classify MTNNO and HLANO separately

In Decision Tree file I pruned out these values from feature set - these all come from the language section:
- MTNEN (mother tongue english) - high correlation to classified feature in tree
- MTNFR (mother tongue french)   - ditto
- HLAFR (home language french)  - ditto
- HLAEN (home language english) - ditto
- HLBEN (home language part b english)
- HLBFR (home language part b french) 
- HLBNO 
- NOL (knowledge of non-official languages) - very high correlation to classified features
- FOL (first official language spoken) 
- KOL (knowledge of official languages) 

_----the below language features had very low influence in the decision tree----_

- LWAEN (language at work part a english)
- LWAFR (language at work part a french)
- LWANO (language at work part a first write-in component)
- LWBEN (language at work part b english)
- LWBFR (language at work part b french)
- LWBNO (language at work part b first write-in component)

In [5]:
# class label: home language part A - first language write in component
homeLang = dataset.iloc[:,65]
# class label: mother tongue part A - first language write in component
motherTongue = dataset.iloc[:, 96]
# variables
x1, x2, x3 = dataset.iloc[:, 18:65], dataset.iloc[:, 66:96], dataset.iloc[:, 97:-1]
x = pd.concat([x1,x2,x3], axis=1, sort=False)
weights = dataset.iloc[:, 1]

In [6]:
x

,ABOID,AGEGRP,AGEIMM,ATTSCH,BedRm,BFNMEMB,CapGn,CFInc,CFInc_AT,CfSize,...,SSGRAD,Subsidy,Tenur,TotInc,TotInc_AT,VALUE,VisMin,Wages,WKSWRK,WRKACT
0,6,11,99,1,5,0,99999999,30,27,4,...,5,9,1,97000,73000,450000,13,95000,6,11
1,6,5,99,9,5,0,99999999,30,27,4,...,99,9,1,99999999,99999999,440000,13,99999999,9,99
2,6,2,99,9,5,0,99999999,30,27,4,...,99,9,1,99999999,99999999,440000,13,99999999,9,99
3,6,12,99,1,4,0,99999999,20,19,4,...,8,9,1,46000,41000,839779,13,19000,6,11
4,6,15,99,1,2,0,99999999,16,15,2,...,6,9,1,30000,26000,60000,13,29000,5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930416,6,17,99,1,4,0,200,31,28,2,...,8,9,1,120000,94000,810000,13,130000,6,11
930417,6,13,99,1,8,0,99999999,30,27,4,...,2,8,8,77000,61000,88888888,13,78000,6,11
930418,6,13,99,1,3,0,99999999,21,17,1,...,6,9,1,80000,62000,710000,13,81000,5,9
930419,6,21,7,1,3,0,99999999,13,13,3,...,1,0,2,22000,23000,99999999,2,99999999,9,1


There are no null values in the dataset

In [7]:
x.isna().sum()

ABOID     0
AGEGRP    0
AGEIMM    0
ATTSCH    0
BedRm     0
         ..
VALUE     0
VisMin    0
Wages     0
WKSWRK    0
WRKACT    0
Length: 120, dtype: int64

Verifying that the metadata is accurate

In [8]:
x[(x['CHDBN'] == 99999999) | (x['CHDBN'] == 8888888)].shape[0]/x.shape[0]

0.8837547733767832

In [9]:
x[x['POB'] == 88].shape

(14478, 120)

Close enough... Taking invalid data seen in metadata and putting in a dictionary

In [10]:
missing_dict = dict(
    AGEGRP=9139,
    MOB1=9884,
    Mob5=51091,
    PR1=188597,
    PR5=88597,
    DPGRSUM=21199,
    ETHDER=31391,
    VisMin=17496,
    HLANO=4081, # This is one of the target columns
    KOL=1754,
    LWAEN=392592,
    LWAFR=392592,
    LWANO=391418,
    LWBEN=392592,
    LWBFR=392592,
    LWBNO=391418,
    MTNEn=1754,
    MTNFr=1754,
    MTNNO=13756, # This is one of the target columns
    NOL=13756,
    AGEIMM=726540,
    CitOth=817447,
    GENSTAT=432,
    IMMCAT5=2412,
    IMMSTAT=2412,
    POB=14478,
    POBF=37281,
    POBM=37281,
    YRIMM=726538,
    ATTSCH=157104,
    CIP2011=157131,
    CIP2011_STEM_SUM=157131,
    HDGREE=157131,
    LOC_ST_RES=157131,
    LOCSTUD=503155,
    SSGRAD=157131,
    COW=391418,
    FPTWK=407453,
    LFACT=157131,
    LSTWRK=157131,
    NAICS=391418,
    NOC16=391418,
    NOCS=391418,
    WKSWRK=391418,
    WRKACT=157131,
    DIST=502619,
    MODE=435003,
    POWST=391418,
    PWDUR=435003,
    PWLEAVE=435003,
    PWOCC=542244,
    PWPR=461131,
    CapGn=844416,
    CFInc=5542,
    CFInc_AT=5542,
    CHDBN=828655,
    ChldC=886332,
    CQPPB=752220,
    EFDecile=5542,
    EfDIMBM=5542,
    EFInc=5542,
    EFInc_AT=5542,
    EICBN=862277,
    EmpIn=384479,
    GovtI=564132,
    GTRfs=399242,
    HHInc=5542,
    HHInc_AT=5542,
    HHMRKINC=5542,
    IncTax=415816,
    Invst=701218,
    LICO=2758,
    LICO_AT=2758,
    LoLIMA=2758,
    LoLIMB=2758,
    LoMBM=2758,
    MrkInc=272659,
    OASGI=795379,
    OtInc=810831,
    Retir=817735,
    SempI=849594,
    TotInc=194147,
    TotInc_AT=193828,
    Wages=427992,
    CfSize=3118,
    EfSize=3118,
    PKID0_1=164769,
    PKID15_24=164769,
    PKID2_5=164769,
    PKID25=164769,
    PKID6_14=164769,
    PKIDS=164769,
    HHSIZE=3118,
    BedRm=8320,
    CONDO=5406,
    DTYPE=2764,
    HCORENEED_IND=45134,
    NOS=8320,
    PresMortG=253489,
    REPAIR=540,
    ROOMS=8320,
    Subsidy=672083,
    Tenur=4849,
    VALUE=258338
)

In [11]:
total_count = x.shape[0]

df_missing = pd.DataFrame(list(missing_dict.items()), columns=['Feature', 'Count'])
df_missing['Ratio'] = df_missing['Count'] / total_count
df_missing

,Feature,Count,Ratio
0,AGEGRP,9139,0.009822
1,MOB1,9884,0.010623
2,Mob5,51091,0.054912
3,PR1,188597,0.202701
4,PR5,88597,0.095222
...,...,...,...
99,REPAIR,540,0.000580
100,ROOMS,8320,0.008942
101,Subsidy,672083,0.722343
102,Tenur,4849,0.005212


In [12]:
df_missing = df_missing.sort_values(by=['Ratio'], ascending=False)
source = ColumnDataSource(data=df_missing)

p = figure(x_range=df_missing['Feature'], plot_height=500, plot_width=2000, title="Missing Ratios",
           toolbar_location=None, x_axis_label='Feature', y_axis_label='Ratio', tools="")
p.vbar(x='Feature', top='Ratio', width=0.9, source=source)

p.xaxis.major_label_orientation = np.pi/4
p.xgrid.grid_line_color = None
p.y_range.start = 0

p.add_tools(HoverTool(tooltips=[('Feature', '@Feature'),
                                ('Ratio', '@Ratio')]))

show(p)

Looking at the above data, a logical point to prune features would be those that are over 30% missing ratio. Proceeding with this option

In [13]:
df_missing_pruned = df_missing[df_missing['Ratio'] > 0.3]
df_missing_pruned

,Feature,Count,Ratio
56,ChldC,886332,0.952614
62,EICBN,862277,0.926760
80,SempI,849594,0.913129
52,CapGn,844416,0.907563
55,CHDBN,828655,0.890624
79,Retir,817735,0.878887
21,CitOth,817447,0.878578
78,OtInc,810831,0.871467
77,OASGI,795379,0.854859
57,CQPPB,752220,0.808473


In [14]:
len(df_missing_pruned)

39

Feature set needs to be further pruned...

In [15]:
df_unpruned = df_missing[df_missing['Ratio'] <= 0.3]
to_impute = df_unpruned['Feature'].tolist()
to_impute

['MrkInc',
 'VALUE',
 'PresMortG',
 'TotInc',
 'TotInc_AT',
 'PR1',
 'PKID0_1',
 'PKID2_5',
 'PKID25',
 'PKID6_14',
 'PKIDS',
 'PKID15_24',
 'CIP2011',
 'CIP2011_STEM_SUM',
 'LFACT',
 'HDGREE',
 'LOC_ST_RES',
 'WRKACT',
 'SSGRAD',
 'LSTWRK',
 'ATTSCH',
 'PR5',
 'Mob5',
 'HCORENEED_IND',
 'POBF',
 'POBM',
 'ETHDER',
 'DPGRSUM',
 'VisMin',
 'POB',
 'NOL',
 'MTNNO',
 'MOB1',
 'AGEGRP',
 'BedRm',
 'ROOMS',
 'NOS',
 'HHInc_AT',
 'EFDecile',
 'EfDIMBM',
 'EFInc_AT',
 'EFInc',
 'HHMRKINC',
 'CFInc_AT',
 'CFInc',
 'HHInc',
 'CONDO',
 'Tenur',
 'HLANO',
 'HHSIZE',
 'CfSize',
 'EfSize',
 'DTYPE',
 'LoLIMA',
 'LoLIMB',
 'LICO',
 'LoMBM',
 'LICO_AT',
 'IMMCAT5',
 'IMMSTAT',
 'MTNFr',
 'MTNEn',
 'KOL',
 'REPAIR',
 'GENSTAT']

In [16]:
df_unpruned.shape[0]

65

####  Data Imputation

Need to impute the dummy values used in the data, e.g. the “99999999” seen in the Wages column. The PDF metadata tells you for each column what it used for its dummy values.

Simplest solution: Remove the rows which have these values.

Simple solution: take the average (mean/median/mode for numerical, median/mode for categorical) of the non-dummy values and replace dummy values by the average found.

Ultra fun solution: train a model (like our best friend, XGBoost) to predict the missing values.


#### Going with approach 2: replace dummy values with mean for continuous variables and mode for categorical variables

In [17]:
imputed = pd.read_csv("pumf-98M0001-E-2016-individuals_F1.csv", usecols=to_impute)

In [18]:
na_dict = {
     'MrkInc':[88888888, 99999999],
     'VALUE':[88888888, 99999999],
     'PresMortG':[9],
     'TotInc':[88888888, 99999999],
     'TotInc_AT':[88888888, 99999999],
     'PR1':[99],
     'PKID0_1':[9],
     'PKID15_24':[9],
     'PKID2_5':[9],
     'PKID25':[9],
     'PKIDS':[9],
     'PKID6_14':[9],
     'WRKACT':[99],
     'LOC_ST_RES':[9],
     'HDGREE':[99],
     'CIP2011_STEM_SUM':[99],
     'SSGRAD':[99],
     'CIP2011':[99],
     'LFACT':[99],
     'LSTWRK':[9],
     'ATTSCH':[9],
     'PR5':[99],
     'Mob5':[9],
     'HCORENEED_IND':[888],
     'POBF':[8],
     'POBM':[8],
     'ETHDER':[88],
     'DPGRSUM':[88],
     'VisMin':[88],
     'POB':[88],
     'NOL':[88],
     'MTNNO':[88], # This is one of the target columns
     'MOB1':[9],
     'AGEGRP':[88],
     'BedRm':[8],
     'ROOMS':[88],
     'NOS':[8],
     'EFInc_AT':[88],
     'CFInc_AT':[88],
     'EFDecile':[88],
     'EfDIMBM':[88],
     'CFInc':[88],
     'HHInc':[88],
     'HHInc_AT':[88],
     'HHMRKINC':[88],
     'EFInc':[88],
     'CONDO':[8],
     'Tenur':[8],
     'HLANO':[9], # This is one of the target columns
     'HHSIZE':[8],
     'CfSize':[8],
     'EfSize':[8],
     'DTYPE':[8],
     'LoLIMB':[9],
     'LoMBM':[9],
     'LICO':[9],
     'LICO_AT':[9],
     'LoLIMA':[9],
     'IMMCAT5':[88],
     'IMMSTAT':[8],
     'MTNFr':[8],
     'MTNEn':[8],
     'KOL':[8],
     'REPAIR':[8],
     'GENSTAT':[8]
}

In [19]:
imputed['MrkInc'].value_counts()

 99999999    267117
 1000         19921
 2000         13895
 3000         12954
 1            12610
              ...  
-41000            2
-48000            2
 500000           1
-39000            1
-46000            1
Name: MrkInc, Length: 265, dtype: int64

In [20]:
# Replacing the dummy values with the median for each of the columns
# for col in na_dict.keys():
#     curr = imputed[col]
#     valid = curr
#     for val in na_dict[col]:
#         valid = valid[valid != val]
#     median = valid.median()
# #     print(f'median for {col} is {median}')
#     for val in na_dict[col]:
#         curr[curr == val] = median

In [21]:
# Replacing the dummy values
for col in na_dict.keys():
    curr = imputed[col]
    valid = curr
    vals = na_dict[col]
    if len(vals) > 1: # variable is continuous
        for val in vals:
            valid = valid[valid != val]
        average = int(valid.mean())
    else: # variable is categorical
        valid = valid[valid != vals[0]]
        average = valid.mode().iloc[0]
#     print(f'average for {col} is {average}')
    for val in vals:
        curr[curr == val] = average

In [22]:
imputed['MrkInc'].value_counts()

 49131     272659
 1000       19921
 2000       13895
 3000       12954
 1          12610
            ...  
-47000          2
-41000          2
-39000          1
-46000          1
 500000         1
Name: MrkInc, Length: 264, dtype: int64

### Now that the selected features have been imputed, combine with the original dataset to have preprocessed data for model fitting

In [23]:
all_cols = pd.read_csv("pumf-98M0001-E-2016-individuals_F1.csv", nrows=1).columns
unimputed_cols = list(set(all_cols) - set(missing_dict.keys()))
df_selected = pd.read_csv("pumf-98M0001-E-2016-individuals_F1.csv", usecols=unimputed_cols)
df_selected = pd.concat([df_selected, imputed], axis=1, sort=False).reset_index()

df_selected

,index,PPSORT,WEIGHT,WT1,WT2,WT3,WT4,WT5,WT6,WT7,...,PresMortG,REPAIR,ROOMS,SSGRAD,Tenur,TotInc,TotInc_AT,VALUE,VisMin,WRKACT
0,0,453141,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,1,11,5,1,97000,73000,450000,13,11
1,1,923226,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,1,11,4,1,49071,39759,440000,13,11
2,2,385097,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,1,11,4,1,49071,39759,440000,13,11
3,3,732612,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,2,8,8,1,46000,41000,839779,13,11
4,4,143665,37.120914,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,1,3,5,6,1,30000,26000,60000,13,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930416,930416,700854,37.037277,0.0,0.0,0.000000,0.000000,0.0,592.596438,0.0,...,0,1,11,8,1,120000,94000,810000,13,11
930417,930417,821443,37.037277,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,8,1,5,2,1,77000,61000,494190,13,11
930418,930418,116531,37.042280,0.0,0.0,592.676474,0.000000,0.0,0.000000,0.0,...,1,2,5,6,1,80000,62000,710000,13,9
930419,930419,499993,37.037277,0.0,0.0,0.000000,592.596438,0.0,0.000000,0.0,...,1,2,7,1,2,22000,23000,494190,2,1


### Writing data with pruned and imputed columns to new csv

In [24]:
df_selected.to_csv('pumf-2016-selected-features.csv', index=False)

In [26]:
df_selected.columns.values

array(['index', 'PPSORT', 'WEIGHT', 'WT1', 'WT2', 'WT3', 'WT4', 'WT5',
       'WT6', 'WT7', 'WT8', 'WT9', 'WT10', 'WT11', 'WT12', 'WT13', 'WT14',
       'WT15', 'WT16', 'ABOID', 'BFNMEMB', 'CFSTAT', 'Citizen', 'CMA',
       'DETH123', 'FOL', 'HHTYPE', 'HLAEN', 'HLAFR', 'HLBEN', 'HLBFR',
       'HLBNO', 'MarStH', 'PR', 'PRIHM', 'REGIND', 'Sex', 'SHELCO',
       'AGEGRP', 'ATTSCH', 'BedRm', 'CFInc', 'CFInc_AT', 'CfSize',
       'CIP2011', 'CIP2011_STEM_SUM', 'CONDO', 'DPGRSUM', 'DTYPE',
       'EFDecile', 'EfDIMBM', 'EFInc', 'EFInc_AT', 'EfSize', 'ETHDER',
       'GENSTAT', 'HCORENEED_IND', 'HDGREE', 'HHInc', 'HHInc_AT',
       'HHMRKINC', 'HHSIZE', 'HLANO', 'IMMCAT5', 'IMMSTAT', 'KOL',
       'LFACT', 'LICO', 'LICO_AT', 'LOC_ST_RES', 'LoLIMA', 'LoLIMB',
       'LoMBM', 'LSTWRK', 'MOB1', 'Mob5', 'MrkInc', 'MTNEn', 'MTNFr',
       'MTNNO', 'NOL', 'NOS', 'PKID0_1', 'PKID15_24', 'PKID2_5', 'PKID25',
       'PKID6_14', 'PKIDS', 'POB', 'POBF', 'POBM', 'PR1', 'PR5',
       'PresMortG', 'REPAIR